In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd gdrive/MyDrive/SPARSe

/content/gdrive/MyDrive/SPARSe


In [4]:
!pip install SQLAlchemy==1.3.18 PyYAML==6.0 psycopg2-binary==2.9.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.0
    Uninstalling SQLAlchemy-2.0.0:
      Successfully uninstalled SQLAlchemy-2.0.0


In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [17]:
projects = pd.read_csv("./data/projects.csv")[["projectid", "date_posted"]]

In [39]:
from datetime import datetime, timedelta

In [20]:
projects['date_posted'] = projects.date_posted.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [21]:
projects.sort_values('date_posted', ascending=False)

,projectid,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12
1,90de744e368a7e4883223ca49318ae30,2014-05-12
192,89d557120f75b8c8e69bfd34f3e77321,2014-05-11
199,16dfbf8580a876966f2e901b9b318026,2014-05-11
198,2bb856e43c4f0da2d5c6d34e9dcfa33e,2014-05-11
...,...,...
664077,7bb427eabf9b9a88a4e2d592c656528a,2002-09-17
664078,3a0c7c8ec39415e28d1d0eccf6578ccd,2002-09-17
664095,82e536f14eadf2671a70e03416f695a3,2002-09-16
664096,e139df754a873a62d93daa56acbf8040,2002-09-13


In [22]:
projects.head()

,projectid,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,2014-05-12
1,90de744e368a7e4883223ca49318ae30,2014-05-12
2,32943bb1063267de6ed19fc0ceb4b9a7,2014-05-11
3,bb18f409abda2f264d5acda8cab577a9,2014-05-11
4,24761b686e18e5eace634607acbcc19f,2014-05-11


In [91]:
last_donation_date = datetime(2014, 4, 28)
last_date = last_donation_date - timedelta(weeks=16)
last_date -= timedelta(weeks=6)

projects = projects.loc[projects['date_posted'] <= last_date]

idx = 0      
final_dataset_date = projects.iloc[idx]["date_posted"]
day_of_week = (projects.iloc[idx]["date_posted"].weekday() + 1) % 7
end_date = final_dataset_date - timedelta(days=day_of_week)
print(end_date)

2013-11-24 00:00:00


In [92]:
train_valid_test_ids = {}

end_test = end_date
for train_val_id in range(3, 0, -1):
  start_test = end_test - timedelta(weeks = 6)
  start_val = start_test - timedelta(weeks = 10)
  start_train = start_val - timedelta(weeks = 36)

  test_set = projects.loc[projects['date_posted'] > start_test].loc[projects['date_posted'] <= end_test]
  valid_set = projects.loc[projects['date_posted'] > start_val].loc[projects['date_posted'] <= start_test]
  train_set = projects.loc[projects['date_posted'] > start_train].loc[projects['date_posted'] <= start_val]

  print(f"Train set {train_val_id}", train_set.sort_values('date_posted', ascending=True))
  print(f"Valid set {train_val_id}", valid_set.sort_values('date_posted', ascending=True))
  print(f"Test set {train_val_id}", test_set.sort_values('date_posted', ascending=True))


  train_valid_test_ids[f"train{train_val_id}"] = train_set["projectid"].tolist()
  train_valid_test_ids[f"valid{train_val_id}"] = valid_set["projectid"].tolist()
  train_valid_test_ids[f"test{train_val_id}"] = test_set["projectid"].tolist()

  end_test -= timedelta(weeks = 10)

print(len(train_valid_test_ids["test3"]))

Train set 3                                projectid date_posted
186628  3e8a7f17218e67efc1e9871c41d3f8e6  2012-11-26
186519  82e2fde263aaeac39886c3692b56a607  2012-11-26
186518  03d3c31733d0bf7214c06d7f01be5add  2012-11-26
186517  95d18b6a2dc1fbd284dfe1f24a7cf5d3  2012-11-26
186516  4505b7d959518c7981a3d733830a5338  2012-11-26
...                                  ...         ...
123446  a5f167757c392b4d0d3bcaf783469738  2013-08-04
123445  a28cb476d145f51cbcbd5754a9f93d87  2013-08-04
123444  91cc31a4670b39b16d84919f3068f4e7  2013-08-04
123442  f017f20c2ac0dd219ac1dfd25b1b7a75  2013-08-04
123002  b2996d650cfb8ad54a5684a81f0a07c6  2013-08-04

[63627 rows x 2 columns]
Valid set 3                                projectid date_posted
123001  55000b34a9fba27095e24142a63bcfd3  2013-08-05
122705  be3dad137a7b32085a649248e2fdb9f8  2013-08-05
122706  9e00acc781df8876241e477c561a1f8b  2013-08-05
122707  5cd1560042355f331d105eb1111914d1  2013-08-05
122708  d079d289719180078e1edd8b9f5ec472  2013-08

In [93]:
import json
with open("./update2_baseline_3trainvalsplit-enddate_2013_11_24-10wk.json", 'w') as f:
  json.dump(train_valid_test_ids, f)